In [46]:
# Pip Installs

#pip install mysql-connector-python
#pip install pyodbc
#pip install sqlalchemy

In [47]:
from flask import Flask, request, jsonify
import joblib
import pandas as pd
import numpy as np
from sklearn.preprocessing import MinMaxScaler
from statsmodels.tsa.stattools import grangercausalitytests, adfuller
from statsmodels.tsa.statespace.varmax import VARMAX
from statsmodels.tsa.api import VAR
import os

from sklearn.metrics import mean_squared_error
from statistics import mean
import math
from darts.models import KalmanForecaster
from darts import TimeSeries
# Azure database connectors
import mysql.connector
from mysql.connector import errorcode
import pyodbc
import pandas as pd
from sqlalchemy import create_engine, text

def load_data():
    server = 'dscpserver.database.windows.net'
    database = 'DSCPDatabase'
    username = 'admindscp'
    password = 'T01b26121gH'
    driver = 'ODBC Driver 17 for SQL Server'

    connection_string = f'mssql+pyodbc://{username}:{password}@{server}/{database}?driver={driver.replace(" ", "+")}'

    # Create an engine object
    engine = create_engine(connection_string)

    # Query the database and load into DataFrame
    query = "SELECT * FROM [dbo].[devicedata]"
    df = pd.read_sql(query, engine)

    # Display the DataFrame
    df.head()

    return df

In [48]:
#run_once_data = load_data()

#new_data = run_once_data.copy()


data = pd.read_csv('query.csv')
data.head()


,devicename,sensor,deviceid,value,dbtimestamp
0,NDS011,7 in 1 Soil Sensor Potassium,12,519.0,2024-07-30T18:48:30.9352050
1,NDS011,7 in 1 Soil Sensor Phosphorus,12,522.0,2024-07-30T18:48:30.9254490
2,NDS011,7 in 1 Soil Sensor Nitrogen,12,206.0,2024-07-30T18:48:30.9212790
3,NDS011,7 in 1 Soil Sensor PH,12,4.6,2024-07-30T18:48:30.9169920
4,NDS011,7 in 1 Soil Sensor EC,12,1173.0,2024-07-30T18:48:30.9074620


In [49]:
def converting_format(data):
    # Convert the 'dbtimestamp' column to datetime
    data['dbtimestamp'] = pd.to_datetime(data['dbtimestamp'])

    # Round the timestamps to the nearest 5 seconds
    data['dbtimestamp'] = data['dbtimestamp'].dt.round('5s')

    # Pivot the data, using 'dbtimestamp' as index, 'sensor' as columns, and 'value' as values
    pivoted_data = data.pivot_table(index='dbtimestamp', columns='sensor', values='value', aggfunc='mean')

    # Forward fill the missing values
    pivoted_data_forward_filled = pivoted_data.ffill()

    # Backward fill the missing values
    pivoted_data_filled = pivoted_data_forward_filled.bfill()
    pivoted_data_filled.index = pivoted_data_filled.index + pd.Timedelta(hours=8)
    pivoted_data_filled.columns = [col for col in pivoted_data_filled.columns]
    pivoted_data_filled = pivoted_data_filled.reset_index()
    pivoted_data_filled.rename(columns={
        'dbtimestamp': 'TimeFrame',
        '7 in 1 Soil Sensor Potassium': 'Potassium',
        '7 in 1 Soil Sensor Phosphorus': 'Phosphorus',
        '7 in 1 Soil Sensor Nitrogen': 'Nitrogen',
        '7 in 1 Soil Sensor PH': 'PH',
        '7 in 1 Soil Sensor EC': 'EC',
        '7 in 1 Soil Sensor Temperature': 'temperature',
        '7 in 1 Soil Sensor Moisture': 'mositure',
        'SCD41 Humidity': 'humidity',
        'SCD41 Temperature': 'SCD41 Temperature',
        'SCD41 CO2': 'CO2'
    }, inplace=True)
    pivoted_data_5min = pivoted_data_filled.set_index('TimeFrame').resample('5T').mean().reset_index()

    return pivoted_data_5min

In [50]:
new_data = converting_format(data)
new_data.tail(10)

C:\Users\quek3\AppData\Local\Temp\ipykernel_37792\2755520435.py:32: FutureWarning: 'T' is deprecated and will be removed in a future version, please use 'min' instead.
  pivoted_data_5min = pivoted_data_filled.set_index('TimeFrame').resample('5T').mean().reset_index()


,TimeFrame,EC,mositure,Nitrogen,PH,Phosphorus,Potassium,temperature,CO2,humidity,SCD41 Temperature
524,2024-07-31 02:00:00,1179.181818,28.227273,207.454545,4.600000,525.454545,521.454545,29.554545,492.454545,96.402339,28.789196
525,2024-07-31 02:05:00,1178.272727,28.563636,207.090909,4.600000,525.000000,521.181818,29.500000,492.909091,95.507466,28.737247
526,2024-07-31 02:10:00,1179.333333,28.458333,207.416667,4.608333,525.500000,521.500000,29.525000,471.000000,94.762975,28.615403
527,2024-07-31 02:15:00,1176.181818,28.236364,206.818182,4.600000,524.000000,520.000000,29.500000,466.727273,94.973816,28.642814
528,2024-07-31 02:20:00,1175.083333,28.433333,206.583333,4.750000,523.416667,519.583333,29.500000,488.000000,96.115308,28.750254
529,2024-07-31 02:25:00,1171.636364,27.663636,205.909091,4.690909,522.000000,518.454545,29.500000,506.363636,96.143628,28.752540
530,2024-07-31 02:30:00,1173.300000,28.000000,206.100000,4.940000,522.400000,518.900000,29.500000,490.500000,95.436943,28.667430
531,2024-07-31 02:35:00,1170.750000,29.450000,205.416667,4.700000,521.250000,517.666667,29.500000,470.666667,94.625518,28.552205
532,2024-07-31 02:40:00,1172.181818,28.063636,205.727273,4.609091,521.818182,518.363636,29.490909,460.545455,95.240571,28.552023
533,2024-07-31 02:45:00,1172.250000,29.125000,205.750000,4.600000,521.875000,518.375000,29.487500,481.750000,95.774015,28.667363


In [51]:
#model_path = 'C:/Users/marke/DSCP/models/test_forecast_model.pkl'

#def load_model():
#    return joblib.load(model_path)

#def save_model(model):
#    joblib.dump(model, model_path)

def detect_outliers(series, threshold=2): # need to set threshold of 1 for changes to be made
    mean = series.mean()
    std = series.std()
    z_scores = (series - mean) / std
    return z_scores.abs() > threshold

def extract_p_values(test_results, max_lag): # for granger causality test
    p_values = {}
    for lag in range(1, max_lag + 1):
        p_values[lag] = test_results[lag][0]['ssr_ftest'][1]    
    return p_values

def data_preprocessing(new_data):

    ### Load new data
    #new_data = pd.read_csv('C:/Users/marke/DSCP/compost_data/fromazure (Macpherson).csv')

    ### Fixing Data Types and Index
    # new_data['TimeFrame'] = pd.to_datetime(new_data['TimeFrame'])
    # start_timestamp = new_data['TimeFrame'].min() + pd.Timedelta(minutes=10)

    # new_timestamps = pd.date_range(start=start_timestamp, periods = len(new_data), freq='10T')    # Generate the new timestamps at 10-minute intervals
    # new_df = pd.DataFrame({'TimeFrame': new_timestamps})
    # new_data['TimeFrame'] = new_df['TimeFrame']     # Replace the NaT values with the new timestamps

    new_data = new_data[-432:]

    ### Fixing 0 values
    

    time_series_col = new_data.columns.to_list()

    for i in time_series_col:
        new_data[i] = new_data[i].replace(0, np.nan)
        new_data[i] = new_data[i].bfill()


    ### Fixing Outliers

    for column in new_data[time_series_col].columns:
        outliers = detect_outliers(new_data[column])
        new_data.loc[outliers, column] = np.nan
        new_data[column].fillna(new_data[column].mean(), inplace=True)

    ### Check for constant values
    #### if the value is constant, it would not be useful for the model, so we drop it

    for column in new_data.columns:
        if new_data[column].nunique() == 1:
            new_data.drop(column, axis=1, inplace=True)

    ### Dropping and Splitting
    
    for feature in new_data.columns:
       if new_data[feature].isnull().sum() > 0:
           new_data.drop(feature, axis=1, inplace=True)

    Time_Col = new_data['TimeFrame']
    new_data.drop(['TimeFrame'], axis=1, inplace=True)

    # new_data.drop(['SoilMoisture', 'Compost'], axis=1, inplace=True) # only for macpherson data

    #### Add the Granger Causality 

    phos_significant_features = []
    nitro_significant_features = []
    pot_significant_features = []
    max_lag = 5

    for feature in new_data.columns:
        test_result = grangercausalitytests(new_data[['Phosphorus', feature]], max_lag)
        p_values = extract_p_values(test_result, max_lag)
        if any(p_value < 0.05 for p_value in p_values.values()):
            phos_significant_features.append(feature)    
    
    for feature in new_data.columns:
        test_result = grangercausalitytests(new_data[['Nitrogen', feature]], max_lag)
        p_values = extract_p_values(test_result, max_lag)
        if any(p_value < 0.05 for p_value in p_values.values()):
            nitro_significant_features.append(feature)   

    for feature in new_data.columns:
        test_result = grangercausalitytests(new_data[['Potassium', feature]], max_lag)
        p_values = extract_p_values(test_result, max_lag)
        if any(p_value < 0.05 for p_value in p_values.values()):
            pot_significant_features.append(feature)

    final_significant_features = list(set(phos_significant_features + nitro_significant_features + pot_significant_features + ['Phosphorus', 'Nitrogen', 'Potassium']))

    new_data = new_data[final_significant_features]

    ### Splitting the data

    new_data['TimeFrame'] = Time_Col

    

    train = new_data[:int(0.8*(len(new_data)))]
    test= new_data[int(0.8*(len(new_data))):]

    train_timeframe = train['TimeFrame']    # Save the timeframe index for the forecast model
    test_timeframe = test['TimeFrame']

    train.drop('TimeFrame', axis=1, inplace=True)

    print(train.head())

    ### Scaling

    scaler = MinMaxScaler(feature_range=(0, 1))
    scaled_train = scaler.fit_transform(train)

    col_list = train.columns.to_list()
    scaled_train = pd.DataFrame(scaled_train, columns=col_list)

    scaled_train.index = train_timeframe
    scaled_train.index = scaled_train.index.to_period('10T')

    test.index = test['TimeFrame']
    test.drop('TimeFrame', axis=1, inplace=True)

    return scaled_train, test, train_timeframe, test_timeframe, scaler

In [52]:
scaled_train, test, train_timeframe, test_timeframe, scaler = data_preprocessing(new_data)

C:\Users\quek3\AppData\Local\Temp\ipykernel_37792\1804670807.py:42: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  new_data[i] = new_data[i].replace(0, np.nan)
C:\Users\quek3\AppData\Local\Temp\ipykernel_37792\1804670807.py:43: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  new_data[i] = new_data[i].bfill()
C:\Users\quek3\AppData\Local\Temp\ipykernel_37792\1804670807.py:51: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The beh


Granger Causality
number of lags (no zero) 1
ssr based F test:         F=1.9439  , p=0.1640  , df_denom=428, df_num=1
ssr based chi2 test:   chi2=1.9575  , p=0.1618  , df=1
likelihood ratio test: chi2=1.9531  , p=0.1623  , df=1
parameter F test:         F=1.9439  , p=0.1640  , df_denom=428, df_num=1

Granger Causality
number of lags (no zero) 2
ssr based F test:         F=4.3457  , p=0.0135  , df_denom=425, df_num=2
ssr based chi2 test:   chi2=8.7937  , p=0.0123  , df=2
likelihood ratio test: chi2=8.7050  , p=0.0129  , df=2
parameter F test:         F=4.3457  , p=0.0135  , df_denom=425, df_num=2

Granger Causality
number of lags (no zero) 3
ssr based F test:         F=3.3108  , p=0.0201  , df_denom=422, df_num=3
ssr based chi2 test:   chi2=10.0970 , p=0.0178  , df=3
likelihood ratio test: chi2=9.9801  , p=0.0187  , df=3
parameter F test:         F=3.3108  , p=0.0201  , df_denom=422, df_num=3

Granger Causality
number of lags (no zero) 4
ssr based F test:         F=3.6416  , p=0.0063  

C:\Users\quek3\AppData\Local\Temp\ipykernel_37792\1804670807.py:112: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  train.drop('TimeFrame', axis=1, inplace=True)
C:\Users\quek3\AppData\Local\Temp\ipykernel_37792\1804670807.py:125: FutureWarning: 'T' is deprecated and will be removed in a future version, please use 'min' instead.
  scaled_train.index = scaled_train.index.to_period('10T')
C:\Users\quek3\AppData\Local\Temp\ipykernel_37792\1804670807.py:128: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  test.drop('TimeFrame', axis=1, inplace=True)


In [53]:
scaled_train.index = train_timeframe

In [54]:
def model_retraining(scaled_train, test):

    ## Create the TimeSeries object
    kalman_df = scaled_train.reset_index()
    kalman_df.rename(columns={'index': 'date'}, inplace=True)
    col_list = scaled_train.columns.tolist()
    series = TimeSeries.from_dataframe(kalman_df, 'TimeFrame', col_list)

    # order_checker = VAR(scaled_train.diff()[1:])

    # sorted_order=order_checker.select_order(maxlags=10)
    # optimal_order = sorted_order.aic

    # prediction_model = VARMAX(scaled_train, order=(optimal_order,0),enforce_stationarity= True)
    # prediction_model = prediction_model.fit(disp=False, maxiter=50, method='powell') # nelder-mead optimization method
    
    ### Train the Kalman model
    model = KalmanForecaster(dim_x=series.width)
    model.fit(series)
    return model

In [55]:
model = model_retraining(scaled_train, test)

In [56]:
def model_validation(model):
    n_forecast = len(test)
    predict = model.predict(n_forecast)  # predict past the dataset

    # Check if the predictions are of type TimeSeries
    if isinstance(predict, TimeSeries):
        predictions = predict.pd_dataframe()
    elif isinstance(predict, (np.ndarray, list)):
        try:
            predictions = pd.DataFrame(predict, columns=test.columns)
        except ValueError as e:
            print(f"Error creating DataFrame from predictions: {e}")
            return None
    elif isinstance(predict, pd.DataFrame):
        predictions = predict
    else:
        print(f"Unexpected data type for predictions: {type(predict)}")
        return None

    # Print the first few rows of the predictions
    print("Predictions (raw):")
    print(predictions.head())

    # Ensure the predictions are in the correct shape for inverse transform
    if predictions.isnull().values.any():
        print("Predictions contain NaN values. Check the model and input data.")
        return None

    # Inverse transform the predictions
    try:
        predictionsdf = pd.DataFrame(scaler.inverse_transform(predictions), columns=predictions.columns)
    except Exception as e:
        print(f"Error in inverse transform: {e}")
        return None

    # Print the first few rows of the inverse transformed predictions
    print("Predictions (inverse transformed):")
    print(predictionsdf.head())

    # Ensure the index matches the test timeframe
    if 'TimeFrame' in test.columns:
        forecast_validation = pd.DataFrame(predictionsdf.values, index=test['TimeFrame'], columns=predictions.columns)
    else:
        forecast_validation = pd.DataFrame(predictionsdf.values, index=test.index, columns=predictions.columns)

    return forecast_validation

In [57]:
validation = model_validation(model)

validation

Predictions (raw):
component            Potassium  Nitrogen        EC  Phosphorus
TimeFrame                                                     
2024-07-30 19:35:00   1.007082  1.012190  1.009581    1.008848
2024-07-30 19:40:00   1.009385  1.015971  1.012151    1.011706
2024-07-30 19:45:00   1.008786  1.014833  1.010612    1.010327
2024-07-30 19:50:00   1.009631  1.014896  1.010818    1.010396
2024-07-30 19:55:00   1.012353  1.017395  1.013476    1.012807
Predictions (inverse transformed):
component   Potassium    Nitrogen           EC  Phosphorus
0          514.758111  204.331077  1165.627464  519.001665
1          514.977541  204.485475  1166.130895  519.271317
2          514.920502  204.439002  1165.829494  519.141237
3          515.000995  204.441606  1165.869806  519.147731
4          515.260380  204.543623  1166.390596  519.375234


component,Potassium,Nitrogen,EC,Phosphorus
TimeFrame,,,,
2024-07-30 19:35:00,514.758111,204.331077,1165.627464,519.001665
2024-07-30 19:40:00,514.977541,204.485475,1166.130895,519.271317
2024-07-30 19:45:00,514.920502,204.439002,1165.829494,519.141237
2024-07-30 19:50:00,515.000995,204.441606,1165.869806,519.147731
2024-07-30 19:55:00,515.260380,204.543623,1166.390596,519.375234
...,...,...,...,...
2024-07-31 02:25:00,554.207862,221.336035,1246.395729,557.550158
2024-07-31 02:30:00,554.858524,221.616353,1247.734290,558.192590
2024-07-31 02:35:00,555.512482,221.898092,1249.079642,558.838291


In [58]:
def minimum_score(forecast_val, test):

    phos_valid = None
    nitro_valid = None
    potas_valid = None

    rmse_phos=math.sqrt(mean_squared_error(forecast_val['Phosphorus'],test['Phosphorus']))
    print('Mean value of Phosphorus is : {}. Root Mean Squared Error is :{}'.format(test['Phosphorus'].mean(),rmse_phos))

    rmse_nitro=math.sqrt(mean_squared_error(forecast_val['Nitrogen'],test['Nitrogen']))
    print('Mean value of Nitrogen is : {}. Root Mean Squared Error is :{}'.format(test['Nitrogen'].mean(),rmse_nitro))

    rmse_potas=math.sqrt(mean_squared_error(forecast_val['Potassium'],test['Potassium']))
    print('Mean value of Potassium is : {}. Root Mean Squared Error is :{}'.format(test['Potassium'].mean(),rmse_potas))

    min_score_dict = {'Phos_Score': 100, 'Nitro_score': 70, 'Potas_score': 200}  ### Set the minimum score for each nutrient 

    if rmse_phos < min_score_dict['Phos_Score']:
        phos_valid = True
    if rmse_nitro < min_score_dict['Nitro_score']:
        nitro_valid = True
    if rmse_potas < min_score_dict['Potas_score']:
        potas_valid = True

    if phos_valid == True and nitro_valid == True and potas_valid == True:
        return True
    else:
        return False

In [59]:
minimum_score(validation, test)

Mean value of Phosphorus is : 519.9614498209326. Root Mean Squared Error is :19.666234566555495
Mean value of Nitrogen is : 204.82263260366707. Root Mean Squared Error is :8.631992493765736
Mean value of Potassium is : 516.0763619841207. Root Mean Squared Error is :19.909675608105196


True

In [60]:
def final_prediction(model, test, scaler, n_forecast=12):
    predict = model.predict(n_forecast)  # predict future values

    # Check if the predictions are of type TimeSeries
    if isinstance(predict, TimeSeries):
        predictions = predict.pd_dataframe()
    elif isinstance(predict, (np.ndarray, list)):
        try:
            predictions = pd.DataFrame(predict, columns=test.columns)
        except ValueError as e:
            print(f"Error creating DataFrame from predictions: {e}")
            return None
    elif isinstance(predict, pd.DataFrame):
        predictions = predict
    else:
        print(f"Unexpected data type for predictions: {type(predict)}")
        return None

    # Print the first few rows of the predictions
    print("Predictions (raw):")
    print(predictions.head())

    # Ensure the predictions are in the correct shape for inverse transform
    if predictions.isnull().values.any():
        print("Predictions contain NaN values. Check the model and input data.")
        return None

    # Inverse transform the predictions
    try:
        predictionsdf = pd.DataFrame(scaler.inverse_transform(predictions), columns=predictions.columns)
    except Exception as e:
        print(f"Error in inverse transform: {e}")
        return None

    # Print the first few rows of the inverse transformed predictions
    print("Predictions (inverse transformed):")
    print(predictionsdf.head())
    
    # Generate new index for the future dates
    start_timestamp = test.index.max() + pd.Timedelta(minutes=10)
    new_index = pd.date_range(start=start_timestamp, periods=n_forecast + 1, freq='10T')[1:]
    
    forecastdf = pd.DataFrame(predictionsdf.values, index=new_index, columns=predictions.columns)


    return forecastdf

In [61]:
print(test)

                      Potassium    Nitrogen           EC  Phosphorus
TimeFrame                                                           
2024-07-30 19:35:00  510.846154  202.384615  1157.076923  514.846154
2024-07-30 19:40:00  510.000000  202.200000  1155.400000  514.000000
2024-07-30 19:45:00  511.545455  202.909091  1158.545455  515.454545
2024-07-30 19:50:00  511.833333  203.000000  1159.250000  515.833333
2024-07-30 19:55:00  512.000000  202.909091  1159.363636  516.000000
...                         ...         ...          ...         ...
2024-07-31 02:25:00  518.454545  205.909091  1171.636364  522.000000
2024-07-31 02:30:00  518.900000  206.100000  1173.300000  522.400000
2024-07-31 02:35:00  517.666667  205.416667  1170.750000  521.250000
2024-07-31 02:40:00  518.363636  205.727273  1172.181818  521.818182
2024-07-31 02:45:00  518.375000  205.750000  1172.250000  521.875000

[87 rows x 4 columns]


In [62]:
final_forecast_df = final_prediction(model,test,scaler, n_forecast=12)
final_forecast_df.head(12)

Predictions (raw):
component            Potassium  Nitrogen        EC  Phosphorus
TimeFrame                                                     
2024-07-30 19:35:00   1.007082  1.012190  1.009581    1.008848
2024-07-30 19:40:00   1.009385  1.015971  1.012151    1.011706
2024-07-30 19:45:00   1.008786  1.014833  1.010612    1.010327
2024-07-30 19:50:00   1.009631  1.014896  1.010818    1.010396
2024-07-30 19:55:00   1.012353  1.017395  1.013476    1.012807
Predictions (inverse transformed):
component   Potassium    Nitrogen           EC  Phosphorus
0          514.758111  204.331077  1165.627464  519.001665
1          514.977541  204.485475  1166.130895  519.271317
2          514.920502  204.439002  1165.829494  519.141237
3          515.000995  204.441606  1165.869806  519.147731
4          515.260380  204.543623  1166.390596  519.375234


C:\Users\quek3\AppData\Local\Temp\ipykernel_37792\343547370.py:41: FutureWarning: 'T' is deprecated and will be removed in a future version, please use 'min' instead.
  new_index = pd.date_range(start=start_timestamp, periods=n_forecast + 1, freq='10T')[1:]


component,Potassium,Nitrogen,EC,Phosphorus
2024-07-31 03:05:00,514.758111,204.331077,1165.627464,519.001665
2024-07-31 03:15:00,514.977541,204.485475,1166.130895,519.271317
2024-07-31 03:25:00,514.920502,204.439002,1165.829494,519.141237
2024-07-31 03:35:00,515.000995,204.441606,1165.869806,519.147731
2024-07-31 03:45:00,515.260380,204.543623,1166.390596,519.375234
2024-07-31 03:55:00,515.579559,204.686891,1167.075376,519.686176
2024-07-31 04:05:00,515.879871,204.822764,1167.705545,519.977258
2024-07-31 04:15:00,516.159073,204.944661,1168.270213,520.239218
2024-07-31 04:25:00,516.442330,205.065422,1168.837080,520.501799
2024-07-31 04:35:00,516.743584,205.194043,1169.445516,520.783764


In [63]:
def parse_ratio(ratio_str):
    """Parses a ratio string (e.g., '1:4:2') into a list of numeric values."""
    return list(map(float, ratio_str.split(':')))

def check_ratio(row, ratio, tolerance=0.45):
    # checks if the values in the row follow the specified ratio within the given tolerance.
    # Normalize the row values and the ratio
    row = row / row.sum()
    ratio = np.array(ratio) / np.sum(ratio)
    
    # Check if the normalized row is within the tolerance of the normalized ratio
    return np.all(np.abs(row - ratio) <= tolerance)

def apply_ratio_check(df, ratio_str, tolerance=0.45):
    # ratio check to each row in the DataFrame
    ratio = parse_ratio(ratio_str)
    return df.apply(check_ratio, axis=1, ratio=ratio, tolerance=tolerance)

In [64]:
def highlight_ratio_check(row):
    if row['RatioCheck'] == False:
        return ['color: red']*len(row)
    if row['RatioCheck'] == True:
        return ['color: green']*len(row)


In [65]:
from flask import Flask, request, render_template, jsonify

app = Flask(__name__)

# post request

@app.route('/') # use POST request if person needs to upload data
def retrain():
    scaled_train, test, train_timeframe, test_timeframe, scaler = data_preprocessing(new_data)
    scaled_train.index = train_timeframe

    model = model_retraining(scaled_train, test)
    score_df = model_validation(model)

    # Quality control
    if minimum_score(score_df, test) == True: # satisfy the minimum score

        # Setting output coloring
        user_ratio = '1:4:2' # example for now

        final_forecast_df = final_prediction(model,test,scaler, n_forecast=12)
        final_forecast_df = final_forecast_df[["Nitrogen", "Phosphorus", "Potassium"]]
        final_forecast_df['RatioCheck'] = apply_ratio_check(final_forecast_df, user_ratio)


        forecast_html = final_forecast_df.to_html(classes = 'table table-striped', table_id = 'forecast_table', justify = 'center')
    
        # html template should have two outputs, one is the dataset, other is the message
        return forecast_html
    #render_template(output1 = o1, 'home.html', query='Model retrained successfully and passed the minimum score')
    #else:  # failed the minimum score
    #    return render_template('home.html', query='Model retrained successfully but failed the minimum score')
    

app.run()

 * Serving Flask app '__main__'
 * Debug mode: off


 * Running on http://127.0.0.1:5000
Press CTRL+C to quit
C:\Users\quek3\AppData\Local\Temp\ipykernel_37792\1804670807.py:42: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  new_data[i] = new_data[i].replace(0, np.nan)
C:\Users\quek3\AppData\Local\Temp\ipykernel_37792\1804670807.py:43: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  new_data[i] = new_data[i].bfill()
C:\Users\quek3\AppData\Local\Temp\ipykernel_37792\1804670807.py:51: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series th


Granger Causality
number of lags (no zero) 1
ssr based F test:         F=1.9439  , p=0.1640  , df_denom=428, df_num=1
ssr based chi2 test:   chi2=1.9575  , p=0.1618  , df=1
likelihood ratio test: chi2=1.9531  , p=0.1623  , df=1
parameter F test:         F=1.9439  , p=0.1640  , df_denom=428, df_num=1

Granger Causality
number of lags (no zero) 2
ssr based F test:         F=4.3457  , p=0.0135  , df_denom=425, df_num=2
ssr based chi2 test:   chi2=8.7937  , p=0.0123  , df=2
likelihood ratio test: chi2=8.7050  , p=0.0129  , df=2
parameter F test:         F=4.3457  , p=0.0135  , df_denom=425, df_num=2

Granger Causality
number of lags (no zero) 3
ssr based F test:         F=3.3108  , p=0.0201  , df_denom=422, df_num=3
ssr based chi2 test:   chi2=10.0970 , p=0.0178  , df=3
likelihood ratio test: chi2=9.9801  , p=0.0187  , df=3
parameter F test:         F=3.3108  , p=0.0201  , df_denom=422, df_num=3

Granger Causality
number of lags (no zero) 4
ssr based F test:         F=3.6416  , p=0.0063  

C:\Users\quek3\AppData\Local\Temp\ipykernel_37792\1804670807.py:112: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  train.drop('TimeFrame', axis=1, inplace=True)
C:\Users\quek3\AppData\Local\Temp\ipykernel_37792\1804670807.py:125: FutureWarning: 'T' is deprecated and will be removed in a future version, please use 'min' instead.
  scaled_train.index = scaled_train.index.to_period('10T')
C:\Users\quek3\AppData\Local\Temp\ipykernel_37792\1804670807.py:128: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  test.drop('TimeFrame', axis=1, inplace=True)
C:\Users\quek3\AppData\Local\Temp\ipykernel_37792\343547370.py:41: FutureWarn

Predictions (raw):
component            Potassium  Nitrogen        EC  Phosphorus
TimeFrame                                                     
2024-07-30 19:35:00   1.007082  1.012190  1.009581    1.008848
2024-07-30 19:40:00   1.009385  1.015971  1.012151    1.011706
2024-07-30 19:45:00   1.008786  1.014833  1.010612    1.010327
2024-07-30 19:50:00   1.009631  1.014896  1.010818    1.010396
2024-07-30 19:55:00   1.012353  1.017395  1.013476    1.012807
Predictions (inverse transformed):
component   Potassium    Nitrogen           EC  Phosphorus
0          514.758111  204.331077  1165.627464  519.001665
1          514.977541  204.485475  1166.130895  519.271317
2          514.920502  204.439002  1165.829494  519.141237
3          515.000995  204.441606  1165.869806  519.147731
4          515.260380  204.543623  1166.390596  519.375234
Mean value of Phosphorus is : 519.9614498209326. Root Mean Squared Error is :19.666234566555495
Mean value of Nitrogen is : 204.82263260366707. Root Me

C:\Users\quek3\AppData\Local\Temp\ipykernel_37792\1804670807.py:42: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  new_data[i] = new_data[i].replace(0, np.nan)
C:\Users\quek3\AppData\Local\Temp\ipykernel_37792\1804670807.py:43: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  new_data[i] = new_data[i].bfill()
C:\Users\quek3\AppData\Local\Temp\ipykernel_37792\1804670807.py:51: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The beh


Granger Causality
number of lags (no zero) 1
ssr based F test:         F=1.9439  , p=0.1640  , df_denom=428, df_num=1
ssr based chi2 test:   chi2=1.9575  , p=0.1618  , df=1
likelihood ratio test: chi2=1.9531  , p=0.1623  , df=1
parameter F test:         F=1.9439  , p=0.1640  , df_denom=428, df_num=1

Granger Causality
number of lags (no zero) 2
ssr based F test:         F=4.3457  , p=0.0135  , df_denom=425, df_num=2
ssr based chi2 test:   chi2=8.7937  , p=0.0123  , df=2
likelihood ratio test: chi2=8.7050  , p=0.0129  , df=2
parameter F test:         F=4.3457  , p=0.0135  , df_denom=425, df_num=2

Granger Causality
number of lags (no zero) 3
ssr based F test:         F=3.3108  , p=0.0201  , df_denom=422, df_num=3
ssr based chi2 test:   chi2=10.0970 , p=0.0178  , df=3
likelihood ratio test: chi2=9.9801  , p=0.0187  , df=3
parameter F test:         F=3.3108  , p=0.0201  , df_denom=422, df_num=3

Granger Causality
number of lags (no zero) 4
ssr based F test:         F=3.6416  , p=0.0063  

C:\Users\quek3\AppData\Local\Temp\ipykernel_37792\1804670807.py:112: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  train.drop('TimeFrame', axis=1, inplace=True)
C:\Users\quek3\AppData\Local\Temp\ipykernel_37792\1804670807.py:125: FutureWarning: 'T' is deprecated and will be removed in a future version, please use 'min' instead.
  scaled_train.index = scaled_train.index.to_period('10T')
C:\Users\quek3\AppData\Local\Temp\ipykernel_37792\1804670807.py:128: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  test.drop('TimeFrame', axis=1, inplace=True)
C:\Users\quek3\AppData\Local\Temp\ipykernel_37792\343547370.py:41: FutureWarn

Predictions (raw):
component            Potassium  Nitrogen        EC  Phosphorus
TimeFrame                                                     
2024-07-30 19:35:00   1.007082  1.012190  1.009581    1.008848
2024-07-30 19:40:00   1.009385  1.015971  1.012151    1.011706
2024-07-30 19:45:00   1.008786  1.014833  1.010612    1.010327
2024-07-30 19:50:00   1.009631  1.014896  1.010818    1.010396
2024-07-30 19:55:00   1.012353  1.017395  1.013476    1.012807
Predictions (inverse transformed):
component   Potassium    Nitrogen           EC  Phosphorus
0          514.758111  204.331077  1165.627464  519.001665
1          514.977541  204.485475  1166.130895  519.271317
2          514.920502  204.439002  1165.829494  519.141237
3          515.000995  204.441606  1165.869806  519.147731
4          515.260380  204.543623  1166.390596  519.375234
